## `pbmc_granulocyte_sorted_3k`

In [1]:
import os
import gc
import h5py
import psutil
import pandas as pd
import scanpy as sc
from scipy import sparse

# Load data

In [2]:
# Set-up the paths
input_dir = '/cellar/users/aklie/data/ml4gland/pbmc_granulocyte_sorted_3k/processed/21Sep23/'
h5_file = os.path.join(input_dir, 'pbmc_granulocyte_sorted_3k_filtered_feature_bc_matrix.h5')
bed_file = os.path.join(input_dir, 'pbmc_granulocyte_sorted_3k_atac_peaks.bed')
output_dir = '/cellar/users/aklie/projects/ML4GLand/use_cases/scBasset/pbmc-granulocyte-sorted-3k_10x-Multiome/processed'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [3]:
# Read peaks and h5ad
peak = pd.read_csv(bed_file, sep='\t', names=['chr','start','end'], comment='#')
ad = sc.read_10x_h5(h5_file, gex_only=False)

/cellar/users/aklie/opt/miniconda3/envs/scbasset/lib/python3.7/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


# Filter low quality cells

In [4]:
ad_rna = ad[:, ad.var['feature_types'] == 'Gene Expression']
ad_atac = ad[:, ad.var['feature_types'] == 'Peaks']
ad_atac.var['chr'] = peak['chr'].values
ad_atac.var['start'] = peak['start'].values
ad_atac.var['end'] = peak['end'].values

# basic stats
sc.pp.filter_cells(ad_rna, min_genes=0)
sc.pp.filter_genes(ad_rna, min_cells=0)
sc.pp.filter_cells(ad_atac, min_genes=0)
sc.pp.filter_genes(ad_atac, min_cells=0)

# a gene need to be expressed in 5% cells
# a peak need to be accessible in 5% cells
thres = int(ad.shape[0]*0.05)
ad_rna = ad_rna[:, ad_rna.var['n_cells'] > thres]
ad_atac = ad_atac[:, ad_atac.var['n_cells'] > thres]

/cellar/users/aklie/opt/miniconda3/envs/scbasset/lib/python3.7/site-packages/ipykernel_launcher.py:3: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  This is separate from the ipykernel package so we can avoid doing imports until
/cellar/users/aklie/opt/miniconda3/envs/scbasset/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:140: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['n_genes'] = number


# Save h5ad for next step

In [5]:
chrs = ['chr'+str(i) for i in range(1,23)] + ['chrX', 'chrY']
ad_atac = ad_atac[:, ad_atac.var['chr'].isin(chrs)]
ad_atac.write(os.path.join(output_dir, 'atac_ad.h5ad'))

/cellar/users/aklie/opt/miniconda3/envs/scbasset/lib/python3.7/site-packages/anndata/_core/anndata.py:1235: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/cellar/users/aklie/opt/miniconda3/envs/scbasset/lib/python3.7/site-packages/anndata/_core/anndata.py:1235: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c


# Preprocess specifically for scBasset

In [3]:
%%bash
source activate scbasset
python /cellar/users/aklie/opt/ml4gland/scBasset/bin/scbasset_preprocess.py \
    --ad_file /cellar/users/aklie/projects/ML4GLand/use_cases/scBasset/pbmc-granulocyte-sorted-3k_10x-Multiome/processed/atac_ad.h5ad \
    --input_fasta /cellar/users/aklie/data/ml4gland/genomes/hg38/hg38.fa \
    --out_path /cellar/users/aklie/projects/ML4GLand/use_cases/scBasset/pbmc-granulocyte-sorted-3k_10x-Multiome/processed

process 0 peaks takes 1.6 s
process 1000 peaks takes 2.4 s
process 2000 peaks takes 3.1 s
process 3000 peaks takes 3.9 s
process 4000 peaks takes 4.6 s
process 5000 peaks takes 5.4 s
process 6000 peaks takes 6.1 s
process 7000 peaks takes 6.9 s
process 8000 peaks takes 7.6 s
process 9000 peaks takes 8.4 s
process 10000 peaks takes 9.1 s
process 11000 peaks takes 9.8 s
process 12000 peaks takes 10.5 s
process 13000 peaks takes 11.2 s
process 14000 peaks takes 11.9 s
process 15000 peaks takes 12.6 s
process 16000 peaks takes 13.4 s
process 17000 peaks takes 14.2 s
process 18000 peaks takes 14.9 s
process 19000 peaks takes 15.6 s
process 20000 peaks takes 16.4 s
process 21000 peaks takes 17.2 s
process 22000 peaks takes 18.0 s
process 23000 peaks takes 18.8 s
process 24000 peaks takes 19.5 s
process 25000 peaks takes 20.3 s
process 26000 peaks takes 21.1 s
process 27000 peaks takes 21.9 s
process 28000 peaks takes 22.6 s
process 29000 peaks takes 23.4 s
process 0 peaks takes 1.7 s
process

```bash

```

# Scratch